<a href="https://colab.research.google.com/github/narayanharsh/Automated-ML-models/blob/main/mental_rag_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langgraph langchain-core langchain-community langchain huggingface_hub sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/2

In [ ]:
from google.colab import userdata
import os
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_core.messages import AIMessage

# Set API token
hf_token = userdata.get('HF_TOKEN')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

# Define graph state
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    age: int
    gender: str
    emotion: str
    query: str
    docs: list
    next: str

# Node: InputCollector
def input_collector(state: GraphState) -> GraphState:
    return state

# Node: ValidateInput
def validate_input(state: GraphState) -> dict:
    if state['age'] > 0 and state['gender'] in ['male', 'female'] and state['emotion']:
        return {"next": "RetrieveDocs"}
    return {"next": END}

# Node: RetrieveDocs
def retrieve_docs(state: GraphState) -> GraphState:
    sample_docs = [
        Document(page_content="Exercise and deep breathing help reduce stress and anxiety."),
        Document(page_content="Sleep is crucial for managing symptoms of depression and ADHD.")
    ]
    splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
    split_docs = splitter.split_documents(sample_docs)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    retriever = vectorstore.as_retriever()
    docs = retriever.get_relevant_documents(state["query"])
    state["docs"] = docs
    return state

# Node: SystemReplyGenerator
# def generate_reply(state: GraphState) -> dict:
#     prompt = f"User (Age: {state['age']}, Gender: {state['gender']}, Emotion: {state['emotion']}): {state['query']}\nBased on this, generate a helpful and empathetic response using the following context: {state['docs']}"
#     llm = HuggingFaceHub(model="google/flan-t5-large")
#     response = llm.invoke(prompt)
#     reply = response.content
#     # Dummy relevance check (real logic should use similarity or other signals)
#     relevant = "stress" in reply.lower() or "help" in reply.lower()
#     state["messages"].append(AIMessage(content=reply))
#     return {"next": "ConversationHandler" if relevant else "RewriteReply", "state": state}

def generate_reply(state: GraphState) -> GraphState:
    prompt = (
        f"User (Age: {state['age']}, Gender: {state['gender']}, Emotion: {state['emotion']}): {state['query']}\n"
        f"Based on this, generate a helpful and empathetic response using the following context: {state['docs']}"
    )
    llm = HuggingFaceHub(
        repo_id="Qwen/Qwen2.5-Coder-32B-Instruct"
    )
    response = llm.invoke(prompt)

    reply = response.content if hasattr(response, "content") else response

    # Add reply to messages
    state["messages"].append(AIMessage(content=reply))

    # Decide relevance (mocked for now, ideally use semantic check or classifier)
    if any(term in reply.lower() for term in ["stress", "focus", "overwhelmed"]):
        state["next"] = "relevant"
    else:
        state["next"] = "irrelevant"

    return state

# Node: RewriteReply
# def rewrite_reply(state: GraphState) -> GraphState:
#     previous_reply = state['messages'][-1].content if state['messages'] else ""
#     prompt = f"Rewrite this reply to be more aligned with the user's emotional state ({state['emotion']}): {previous_reply}"
#     llm = HuggingFaceHub(model="google/flan-t5-large")
#     response = llm.invoke(prompt)
#     rewritten = response.content
#     state["messages"].append(AIMessage(content=rewritten))
#     return state

def rewrite_reply(state: GraphState) -> GraphState:
    previous_reply = state['messages'][-1].content if state['messages'] else ""
    prompt = f"Rewrite this reply to be more aligned with the user's emotional state ({state['emotion']}): {previous_reply}"

    llm = HuggingFaceHub(
        repo_id="Qwen/Qwen2.5-Coder-32B-Instruct"
    )
    response = llm.invoke(prompt)

    reply = response.content if hasattr(response, "content") else response
    state["messages"].append(AIMessage(content=reply))
    state["next"] = "relevant"  # After rewrite, we assume it's now relevant

    return state

# Node: ConversationHandler
def conversation_handler(state: GraphState) -> dict:
    latest = state['messages'][-1].content if state['messages'] else ""
    if any(x in latest.lower() for x in ["exit", "quit", "bye"]):
        return {"next": END}
    return {"next": "FollowUpAnalyzer"}

# Node: FollowUpAnalyzer
def follow_up_analyzer(state: GraphState) -> GraphState:
    return state

# Build the LangGraph
workflow = StateGraph(GraphState)
workflow.add_node("InputCollector", RunnableLambda(input_collector))
workflow.add_node("ValidateInput", RunnableLambda(validate_input))
workflow.add_node("RetrieveDocs", RunnableLambda(retrieve_docs))
workflow.add_node("SystemReplyGenerator", RunnableLambda(generate_reply))
workflow.add_node("RewriteReply", RunnableLambda(rewrite_reply))
workflow.add_node("ConversationHandler", RunnableLambda(conversation_handler))
workflow.add_node("FollowUpAnalyzer", RunnableLambda(follow_up_analyzer))

# Edges
workflow.set_entry_point("InputCollector")
workflow.add_edge("InputCollector", "ValidateInput")
workflow.add_conditional_edges("ValidateInput", lambda x: x["next"])
workflow.add_edge("RetrieveDocs", "SystemReplyGenerator")
workflow.add_conditional_edges("SystemReplyGenerator", lambda x: x["next"])
workflow.add_edge("RewriteReply", "ConversationHandler")
workflow.add_conditional_edges("ConversationHandler", lambda x: x["next"])
workflow.add_edge("FollowUpAnalyzer", "RetrieveDocs")

# Compile
app = workflow.compile()

# Run the graph
# initial_state = {
#     "age": 25,
#     "gender": "female",
#     "emotion": "stress",
#     "query": "I'm feeling very overwhelmed with work and can't focus.",
#     "messages": []
# }

from langchain_core.messages import HumanMessage

initial_state = {
    "age": 25,
    "gender": "female",
    "emotion": "stress",
    "query": "I'm feeling very overwhelmed with work and can't focus.",
    "messages": [],
    "docs": []
}

final_result = app.invoke(initial_state)
print("Final Response:", final_result)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Final Response: {'messages': [AIMessage(content="User (Age: 25, Gender: female, Emotion: stress): I'm feeling very overwhelmed with work and can't focus.\nBased on this, generate a helpful and empathetic response using the following context: [Document(id='9844f978-59ed-4f83-a9c6-f5655932dbbf', metadata={}, page_content='Exercise and deep breathing help reduce stress and anxiety.'), Document(id='95a67991-ab26-4c55-b49f-b1d197ff1e96', metadata={}, page_content='Sleep is crucial for managing symptoms of depression and ADHD.')] To create a response that addresses the user's current emotional state and provides actionable advice.\n: It's completely understandable to feel overwhelmed and stressed, especially when you're juggling a lot of work. Taking a moment to focus on your well-being can make a big difference. One effective way to manage stress is through exercise and deep breathing. Even a short walk or a few minutes of stretching can help clear your mind and reduce anxiety. Additionally